<a href="https://colab.research.google.com/github/shlok-py/Pytorch_Tutorial/blob/main/PyTorch_lesson_7_Optimizing_Model_Params.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Optmizing Model Parameters
Now that we have a model and data it’s time to train, validate and test our model by optimizing its parameters on our data. Training a model is an iterative process; in each iteration the model makes a guess about the output, calculates the error in its guess (loss), collects the derivatives of the error with respect to its parameters (as we saw in the previous lesson), and optimizes these parameters using gradient descent. For a more detailed walkthrough of this process, check out this [video on backpropagation from 3Blue1Brown](https://www.youtube.com/watch?v=tIeHLnjs5U8).

In [2]:
import torch
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor
from torch import nn

In [3]:
train_data = datasets.FashionMNIST(
    root = 'data',
    train = True,
    download = True,
    transform = ToTensor(),
)

test_data = datasets.FashionMNIST(
    root = 'data',
    train = False,
    download = True,
    transform = ToTensor(),
)

train_data_loader = DataLoader(train_data, batch_size = 64)
test_data_loader = DataLoader(test_data, batch_size = 64)

  0%|          | 0/26421880 [00:00<?, ?it/s]

Extracting data/FashionMNIST/raw/train-images-idx3-ubyte.gz to data/FashionMNIST/raw



  0%|          | 0/29515 [00:00<?, ?it/s]

Extracting data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to data/FashionMNIST/raw



  0%|          | 0/4422102 [00:00<?, ?it/s]

Extracting data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to data/FashionMNIST/raw



  0%|          | 0/5148 [00:00<?, ?it/s]

Extracting data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to data/FashionMNIST/raw



In [13]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.Relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512,512),
            nn.ReLU(),
            nn.Linear(512,10)
        )
    def forward(self,x):
        x = self.flatten(x)
        logits = self.Relu_stack(x)
        return logits

In [14]:
model = NeuralNetwork()
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (Relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


## Hyperparameters
Hyperparameters are adjustable parameters that let you control the model optimization process. Different hyperparameter values can impact model training and convergence rates

We define the following hyperparameters for training:


*   Number of Epochs
*   Learning_rate
*   Batch_size



In [6]:
learning_rate = 1e-3
batch_size = 64
epochs = 5


## Optimization Loop
Once we set our hyperparameters, we can then train and optimize our model with an optimization loop. Each iteration of the optimization loop is called an **epoch**.

Each epoch consists of two parts:


*   **The train loop**: iterate over the training dataset and try to converge to optimal parameters.

*   **The Validation/Test Loop** - iterate over the test dataset to check if model performance is improving.


## Loss Function
When presented with some training data, our untrained network is likely not to give the correct answer. **Loss function** measures the degree of dissimilarity of obtained result to the target value, and it is the loss function that we want to minimize during training. To calculate the loss we make a prediction using the inputs of our given data sample and compare it against the true data label value.

Common loss functions include `nn.MSELoss` (Mean Square Error) for regression tasks, and `nn.NLLLoss` (Negative Log Likelihood) for classification. `nn.CrossEntropyLoss` combines `nn.LogSoftmax` and `nn.NLLLoss`.

We pass our model’s output logits to `nn.CrossEntropyLoss`, which will normalize the `logits` and compute the prediction error.

In [7]:
# Initialize the loss function
loss_fn = nn.CrossEntropyLoss()

## Optmizer
Optimization is the process of adjusting model parameters to reduce model error in each training step. **Optimization algorithms** define how this process is performed (in this example we use **Stochastic Gradient Descent**). All optimization logic is encapsulated in the `optimizer` object. Here, we use the SGD optimizer; additionally, there are many different optimizers available in PyTorch such as ADAM and RMSProp, that work better for different kinds of models and data.

We initialize the optimizer by registering the model’s parameters that need to be trained, and passing in the learning rate hyperparameter.

In [8]:
optimizer = torch.optim.SGD(model.parameters(), lr = learning_rate)

Inside the training loop, optimization happens in three steps:
*   Call `optimizer.zero_grad()` to reset the gradients of model parameters. Gradients by default add up; to prevent double-counting, we explicitly zero them at each iteration.
*   Backpropagate the prediction loss with a call to `loss.backward()`. PyTorch deposits the gradients of the loss w.r.t. each parameter.
*   Once we have our gradients, we call `optimizer.step()` to adjust the parameters by the gradients collected in the backward pass.

## Full Implementation
We define `train_loop` that loops over our optimization code, and `test_loop` that evaluates the model’s performance against our test data.

In [9]:
def train_loop(model, dataloader, optimizer, loss_fn):
    size = len(dataloader.dataset)
    for batch, (X,y) in enumerate(dataloader):
        #computer prediction and loss
        pred = model(X)
        loss = loss_fn(pred, y)

        # begin backward propagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [10]:
def test_loop(model, dataloader, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [17]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

epochs = 10
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(model, train_data_loader, optimizer, loss_fn)
    test_loop(model,test_data_loader, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.160159  [   64/60000]
loss: 2.149410  [ 6464/60000]
loss: 2.084510  [12864/60000]
loss: 2.105703  [19264/60000]
loss: 2.066886  [25664/60000]
loss: 1.990116  [32064/60000]
loss: 2.042932  [38464/60000]
loss: 1.941629  [44864/60000]
loss: 1.953718  [51264/60000]
loss: 1.901867  [57664/60000]
Test Error: 
 Accuracy: 55.1%, Avg loss: 1.880484 

Epoch 2
-------------------------------
loss: 1.906016  [   64/60000]
loss: 1.874807  [ 6464/60000]
loss: 1.748621  [12864/60000]
loss: 1.801523  [19264/60000]
loss: 1.702698  [25664/60000]
loss: 1.640642  [32064/60000]
loss: 1.688720  [38464/60000]
loss: 1.562856  [44864/60000]
loss: 1.603769  [51264/60000]
loss: 1.517626  [57664/60000]
Test Error: 
 Accuracy: 62.5%, Avg loss: 1.511957 

Epoch 3
-------------------------------
loss: 1.572412  [   64/60000]
loss: 1.537887  [ 6464/60000]
loss: 1.378300  [12864/60000]
loss: 1.458093  [19264/60000]
loss: 1.353406  [25664/60000]
loss: 1.339865  [32064/600